<a href="https://colab.research.google.com/github/xspronken/bigdataW1/blob/master/week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math as m


''' For conversion from natural units'''
G = 6.67*10**(-11)
c = 3*10**8

''' defining Power Spectral Density S(f) '''
fs = 20 #lower limit
f0 = 215 #upper limit

def S(x):
  s0 = 10**(-49)
  return s0*((4.49*x)**(-56)+0.16*x**(-4.52)+0.52+0.32*x**2)

'''variables'''
SNR = 10
def A2(SNR, dx):
  accumulator = 0
  for i in range (int(fs/dx), int(f0/dx)):
   accumulator += (i*dx)**(-7/6) / (S(i*dx))
  return SNR/(4*accumulator)
A = m.sqrt(A2(SNR, 0.001))

m1 = 10**30/((c**2)/G)
m2 = 10**30/((c**2)/G)
f = 150

M = (m1+m2)
n = m1*m2/(M**2)
t_c = 100
phi_c = 100

params = [t_c, phi_c, M, n]

alpha = [1, 0, (20/9*(743/336 + n*11/4)), -16*m.pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
PN = 1


''' defining the Fourrier transform h(f)'''
def h(f, params, x):
  #defining the waveform's variables
  t_c, phi_c, M , n = params
  v = (m.pi*M*f)**(1/3)
  
  #sum of all coefficients (alpha) in the fourrier phase
  x *= 2
  sum_alpha = 0
  for i in range(x):
    sum_alpha += (alpha[i] * (v**i))
  
  #defining the waveform psi(f)
  psi =  2 * m.pi * f * t_c - phi_c - m.pi/4 + 3/(128*n*v**5) * sum_alpha
  
  return  A * f**(-7/6) * m.e**(1j * psi)


''' Calculating the partial derivative of h(f) '''
def h_partial(f, params, x, dx):   
  dparams = params.copy()
  dparams[x] += dx
  dy = h(f, dparams, PN)-h(f, params, PN)
  return dy/dx

''' Generating the Fischer information matrix '''
def generate_matrix(params, dx):
  
# create an n*n matrix where n is the number of observables
  n = len(params)
  matrix = np.empty([n,n]) 
  
# Start filling the matrix
  for i in range (n):
    for j in range (n):
      
      #start integration
      accumulator = 0
      for k in range(int(fs/dx), int(f0/dx)):
        part_i = h_partial(k*dx, params,i, 0.001)
        part_j = np.conj(h_partial(k*dx, params,j, 0.001))
        accumulator += np.real(part_i * part_j / S(k*dx))
      
      #once integration is done add it to the matrix
      matrix[i][j] = accumulator*4
  
  return matrix


''' Extract the diagonal elements from the inverted Fischer matrix'''

def extract_from_matrix(matrix):
  #invert matrix
  y = np.linalg.inv(matrix) 
  z = []
  
  #fetch the square root of diagonal elements
  for i in range (len(y)):
    z.append(m.sqrt(y[i][i]))
  
  return z

test = extract_from_matrix(generate_matrix(params, 0.001))

In [0]:
test